# 3. Dataset construction

## Import dependencies

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely import geometry
from pyproj import CRS
import geog
import shapely
from shapely.geometry import Point
import shapely.wkt
import matplotlib.pyplot as plt
import gc

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
%config Completer.use_jedi = False

import warnings
warnings.filterwarnings("ignore")

In this part of the study, we merge all the data into complete datasets. What is worth mentioning is that we collect data for 2 cities: Warszawa and Kraków, both for 1km2 grids and 500m buffers around Inpost points (only for GWR). In the case of merging for grids we took an assumption about intersection of grid with poviat (it was enough for grids to intersect with poviat boundaries). Additionally, we count other data for each grid. In turn, in the case of merging for buffers, we also count data for each buffer and also we use data from Inspire (if more than one grid intersects with buffer we take avarage value from those grids).

## Parcel data

In this section, we merge the files for parcel points. Finally, we get a file that contains geolocation data for all available pickup points.

In [2]:
bliska_paczka = pd.read_csv("../datasets/raw_data/bliska_paczka.csv", index_col=0)
bliska_paczka = bliska_paczka.loc[bliska_paczka.available == True, ("brand", "operator", "city", "street", "longitude", "latitude")]
bliska_paczka.brand = bliska_paczka.brand.str.lower()
bliska_paczka.operator = bliska_paczka.operator.str.lower()
bliska_paczka.city = bliska_paczka.city.str.lower()
bliska_paczka.street = bliska_paczka.street.str.lower()
bliska_paczka = bliska_paczka.loc[:, ("brand", "operator", "longitude", "latitude")]

In [3]:
dhl = pd.read_csv("../datasets/raw_data/dhl.csv", index_col=0)
dhl.P_TYPE = dhl.P_TYPE.str.lower() 
dhl = dhl.drop(columns=["ID"])
dhl.columns = ["brand", "latitude", "longitude"]
dhl["operator"] = "dhl"
dhl = dhl[["brand", "operator", "longitude", "latitude"]]

In [4]:
df = pd.concat([bliska_paczka, dhl], axis=0)

In [6]:
df = df.drop(columns=["brand"])
df = gpd.GeoDataFrame(df, geometry=[geometry.Point(xy) for xy in zip(df.longitude, df.latitude)], crs=CRS("epsg:4258"))
df = df.drop(columns=["latitude", "longitude"])

In [9]:
df.to_csv('../datasets/preprocessed_data/pickup_points_by_operator.csv', index = False)

## GUGIK and Inspire data

In the next section, we load data for GUGIK and Inspire. GUGIK data is necessary to obtain polygons for 2 poviats (Warszawa and Kraków). In turn, Inspire data provides us with demographic variables that define the population, both total and distinguished by gender and age for 1 km2 grids. We merge both of the above data sets by location, i.e. grids within and intersecting with poviats remain in our data set. Similarly, we add data for parcels of points according to their presence within poviats.

In [10]:
pov = gpd.read_file("../datasets/raw_data/guigk_pov/Powiaty.shx", encoding='utf-8')
pov = pov.loc[pov.JPT_NAZWA_.isin(["powiat Warszawa", "powiat Kraków"])==True, ("JPT_NAZWA_", "geometry")]
pov = pov.to_crs("epsg:4258")

grids = gpd.read_file("../datasets/raw_data/inspire/PD_STAT_GRID_CELL_2011.shp", encoding='utf-8')
grids = grids[['TOT', 'TOT_0_14', 'TOT_15_64', 'TOT_65__', 'TOT_MALE', 'TOT_FEM',
       'MALE_0_14', 'MALE_15_64', 'MALE_65__', 'FEM_0_14', 'FEM_15_64',
       'FEM_65__', 'FEM_RATIO', 'geometry']]
grids = grids.to_crs("epsg:4258")

pov_grids = gpd.sjoin(grids, pov, how="inner", op="intersects")
pov_grids = pov_grids.drop(columns=["index_right"])
pov_grids = pov_grids.reset_index()
pov_grids = pov_grids.rename(columns={"index":"grid_index"})

In [12]:
pov_grids_sliced = pov_grids[["grid_index", "geometry"]].copy() 
df_pov_grids_sliced = gpd.sjoin(df, pov_grids_sliced, how="right", op="within")
df_pov_grids_sliced = df_pov_grids_sliced.drop(columns=["index_left"])
df_pov_grids_sliced = df_pov_grids_sliced.groupby(["operator", "grid_index"], as_index=False).count()
df_pov_grids_sliced = df_pov_grids_sliced.pivot(index='grid_index', columns='operator').fillna(0).reset_index()
df_pov_grids_sliced.columns = df_pov_grids_sliced.columns.droplevel()
df_pov_grids_sliced.columns = ['grid_index', 'dhl', 'dpd', 'fedex', 'inpost', 'poczta', 'ruch', 'ups']

In [13]:
df_pov_grids = pd.merge(pov_grids, df_pov_grids_sliced, how="left", on="grid_index")
df_pov_grids.columns = df_pov_grids.columns.str.lower()
df_pov_grids = df_pov_grids.fillna(0)
df_pov_grids = df_pov_grids[['grid_index','geometry', 'jpt_nazwa_', 'dhl', 'dpd', 'fedex', 'inpost', 'poczta', 'ruch',
                             'ups', 'tot', 'tot_0_14', 'tot_15_64', 'tot_65__', 'tot_male',
                            'tot_fem', 'male_0_14', 'male_15_64', 'male_65__', 'fem_0_14',
                            'fem_15_64', 'fem_65__', 'fem_ratio']]

In [17]:
del bliska_paczka, dhl, pov_grids, pov_grids_sliced, df_pov_grids_sliced; gc.collect();

## OSM data

Finally, in order to merge OSM (points of interest) variables to the final dataset, we created a function that loads OSM data for voivodeships. Then we count the points of interest for each grid. Thus, we obtain the final dataset for grids.

In [18]:
def merging_vars(data, poviat, path):
    
    # df for concrete poviat
    df_output = data[data.jpt_nazwa_ == poviat]
    
    # loading points of interest
    buildings = gpd.read_file(path + 'gis_osm_buildings_a_free_1.shp')
    landuse = gpd.read_file(path + 'gis_osm_landuse_a_free_1.shp')
    pois_a = gpd.read_file(path + 'gis_osm_pois_a_free_1.shp')
    railways = gpd.read_file(path + 'gis_osm_railways_free_1.shp')
    roads = gpd.read_file(path + 'gis_osm_roads_free_1.shp')
    traffic_a = gpd.read_file(path + 'gis_osm_traffic_a_free_1.shp')
    traffic = gpd.read_file(path + 'gis_osm_traffic_free_1.shp')
    transport_a = gpd.read_file(path + 'gis_osm_transport_a_free_1.shp')
    
    # concrete points from above dfs
    buildings_points = buildings[buildings['type'].isin(['house', 'residential', 'bungalow', 'apartment'])][['osm_id', 'geometry']]
    shop_points = buildings[buildings['type'].isin(['supermarket', 'bakery', 'kiosk', 'mall', 'department_store', 'convenience', 'clothes', 'florist', 'chemist'])][['osm_id', 'geometry']]
    parks_points = landuse[landuse['fclass'] == 'park'][['osm_id', 'geometry']]
    forest_points = landuse[landuse['fclass'] == 'forest'][['osm_id', 'geometry']]
    schools_points = pois_a[pois_a['fclass'].isin(['school', 'playground'])][['osm_id', 'geometry']]
    railways_points = railways[['osm_id', 'geometry']]
    cycleways_points = roads[roads['fclass'] == 'cycleway'][['osm_id', 'geometry']]
    parking_points = traffic_a[traffic_a['fclass'] == 'parking'][['osm_id', 'geometry']]
    crossing_points = traffic[traffic['fclass'] == 'crossing'][['osm_id', 'geometry']]
    bus_stop_points = transport_a[transport_a['fclass'] == 'bus_stop'][['osm_id', 'geometry']]
    
    # unnecessery dfs
    del buildings, landuse, pois_a, railways, roads, traffic_a, traffic, transport_a; gc.collect()
    
    # changing crs
    buildings_points = buildings_points.to_crs("epsg:4258")
    shop_points = shop_points.to_crs("epsg:4258")
    parks_points = parks_points.to_crs("epsg:4258")
    forest_points = forest_points.to_crs("epsg:4258")
    schools_points = schools_points.to_crs("epsg:4258")
    railways_points = railways_points.to_crs("epsg:4258")
    cycleways_points = cycleways_points.to_crs("epsg:4258")
    parking_points = parking_points.to_crs("epsg:4258")
    crossing_points = crossing_points.to_crs("epsg:4258")
    bus_stop_points = bus_stop_points.to_crs("epsg:4258")
    
    # list of dataframes
    list_of_dfs = [buildings_points, shop_points, parks_points, forest_points, schools_points, railways_points,
                  cycleways_points, parking_points, crossing_points, bus_stop_points]
    
    # names of new columns
    names = ['buildings', 'shops', 'parks', 'forests', 'schools', 'railways',
                  'cycleways', 'parkings', 'crossings', 'bus_stops']
    
    # groupby points in a loop
    for i in range(len(list_of_dfs)):
        actual_point = gpd.sjoin(list_of_dfs[i], df_output, how="inner", op="intersects")
        x = actual_point[['osm_id', 'grid_index']].groupby(['grid_index']).count()
        x.rename(columns={"osm_id": names[i]}, inplace=True)
        x.reset_index(inplace = True)
        df_output = df_output.merge(x, on = 'grid_index', how='outer')
        
    df_output.fillna(0, inplace=True)
    df_output.drop(columns = {'jpt_nazwa_'}, inplace = True)

    return df_output

## Saving final dataset for grids

In [19]:
df_krakow = merging_vars(df_pov_grids, 'powiat Kraków', '../datasets/raw_data/osm_malopolskie/')
df_krakow[['tot', 'tot_0_14', 'tot_15_64', 'tot_65__', 'tot_male', 'tot_fem',
       'male_0_14', 'male_15_64', 'male_65__', 'fem_0_14', 'fem_15_64',
       'fem_65__']] = df_krakow[['tot', 'tot_0_14', 'tot_15_64', 'tot_65__', 'tot_male', 'tot_fem',
       'male_0_14', 'male_15_64', 'male_65__', 'fem_0_14', 'fem_15_64',
       'fem_65__']].astype(float)

df_krakow.to_csv('../datasets/preprocessed_data/df_krakow.csv', index = False)
df_krakow.to_file('../datasets/preprocessed_data/df_krakow.shp')
del df_krakow; gc.collect()

0

In [21]:
df_warszawa = merging_vars(df_pov_grids, 'powiat Warszawa', '../datasets/raw_data/osm_mazowieckie/')
df_warszawa[['tot', 'tot_0_14', 'tot_15_64', 'tot_65__', 'tot_male', 'tot_fem',
       'male_0_14', 'male_15_64', 'male_65__', 'fem_0_14', 'fem_15_64',
       'fem_65__']] = df_warszawa[['tot', 'tot_0_14', 'tot_15_64', 'tot_65__', 'tot_male', 'tot_fem',
       'male_0_14', 'male_15_64', 'male_65__', 'fem_0_14', 'fem_15_64',
       'fem_65__']].astype(float)

df_warszawa.to_csv('../datasets/preprocessed_data/df_warszawa.csv', index = False)
df_warszawa.to_file('../datasets/preprocessed_data/df_warszawa.shp')
del df_warszawa; gc.collect()

362

## Dataset for GWR

When constructing the dataset for GWR, we followed the same steps as for the above dataset. The only difference is that we collected data for 500m buffers.

In [22]:
df_geo = df.copy()

In [23]:
df_geo['point_id'] = list(range(0, df_geo.shape[0]))

In [24]:
pov_df_geo = gpd.sjoin(pov, df_geo, how="inner", op="intersects")

In [25]:
pov_df_geo = pov_df_geo.merge(df_geo, on = 'point_id')

In [26]:
pov_df_geo = pov_df_geo.drop(columns = {'geometry_x', 'index_right', 'operator_y'})

In [27]:
pov_df_geo = pov_df_geo.rename(columns={"geometry_y": "geometry", "JPT_NAZWA_": "jpt_nazwa_", "operator_x": "operator"})

In [28]:
inpost_points = pov_df_geo[pov_df_geo['operator'] == 'inpost']

## Creating 500m buffer around each Inpost point

In [29]:
# creating buffer (500m radius)
def buffer(point):

    n_points = 50
    angles = np.linspace(0, 360, n_points)
    radius = 500
    polygon = geog.propagate(point, angles, radius)

    x = polygon.tolist()
    lon = list(list(zip(*x))[0])
    lat = list(list(zip(*x))[1])
    pts = gpd.GeoSeries([Point(x, y) for x, y in zip(lon, lat)])
    poly = geometry.Polygon([[p.x, p.y] for p in pts])
    polyg = gpd.GeoSeries(poly)
    

    return polyg

In [30]:
inpost_points['buffer'] = inpost_points.apply(lambda x: buffer(x['geometry']), axis=1)
inpost_points = inpost_points.rename(columns={"geometry": "center"})

In [31]:
inpost_buffers = gpd.GeoDataFrame(inpost_points, geometry = 'buffer', crs = "epsg:4258")
inpost_buffers = inpost_buffers.reset_index()
inpost_buffers = inpost_buffers.rename(columns={"index":"buffer_index"})

## Parcel points

In [32]:
def merge_points(data_inpost, data_operator, operator):
    new_df = data_operator[data_operator['operator'] == operator]
    new_df = gpd.GeoDataFrame(new_df, geometry = 'geometry', crs = "epsg:4258")
    
    actual_point = gpd.sjoin(new_df, data_inpost, how="inner", op="intersects")
    x = actual_point[['operator_left', 'buffer_index']].groupby(['buffer_index']).count()
    x.rename(columns={"operator_left": operator+'_points'}, inplace=True)
    x.reset_index(inplace = True)
    df_output = data_inpost.merge(x, on = 'buffer_index', how='outer')
    df_output.fillna(0, inplace=True)
    
    return df_output

In [33]:
operators = ['inpost', 'poczta', 'dhl', 'ruch', 'dpd', 'ups', 'fedex']
for operator in operators:
    inpost_buffers = merge_points(inpost_buffers, pov_df_geo, operator)

## Inspire data

In [34]:
buff = gpd.sjoin(inpost_buffers, grids, how="inner", op="intersects")

x = buff[['buffer_index', 'TOT', 'TOT_0_14', 'TOT_15_64', 'TOT_65__', 'TOT_MALE', 'TOT_FEM',
       'MALE_0_14', 'MALE_15_64', 'MALE_65__', 'FEM_0_14', 'FEM_15_64',
       'FEM_65__', 'FEM_RATIO']].groupby(['buffer_index']).mean()

inpost_buffers = inpost_buffers.merge(x, on='buffer_index')

## OSM data

In [35]:
def merge_osm(data, poviat, path):
    
    # df for concrete poviat
    df_output = data[data.jpt_nazwa_ == poviat]
    
    # loading points of interest
    buildings = gpd.read_file(path + 'gis_osm_buildings_a_free_1.shp')
    landuse = gpd.read_file(path + 'gis_osm_landuse_a_free_1.shp')
    pois_a = gpd.read_file(path + 'gis_osm_pois_a_free_1.shp')
    railways = gpd.read_file(path + 'gis_osm_railways_free_1.shp')
    roads = gpd.read_file(path + 'gis_osm_roads_free_1.shp')
    traffic_a = gpd.read_file(path + 'gis_osm_traffic_a_free_1.shp')
    traffic = gpd.read_file(path + 'gis_osm_traffic_free_1.shp')
    transport_a = gpd.read_file(path + 'gis_osm_transport_a_free_1.shp')
    
    # concrete points from above dfs
    buildings_points = buildings[buildings['type'].isin(['house', 'residential', 'bungalow', 'apartment'])][['osm_id', 'geometry']]
    shop_points = buildings[buildings['type'].isin(['supermarket', 'bakery', 'kiosk', 'mall', 'department_store', 'convenience', 'clothes', 'florist', 'chemist'])][['osm_id', 'geometry']]
    parks_points = landuse[landuse['fclass'] == 'park'][['osm_id', 'geometry']]
    forest_points = landuse[landuse['fclass'] == 'forest'][['osm_id', 'geometry']]
    schools_points = pois_a[pois_a['fclass'].isin(['school', 'playground'])][['osm_id', 'geometry']]
    railways_points = railways[['osm_id', 'geometry']]
    cycleways_points = roads[roads['fclass'] == 'cycleway'][['osm_id', 'geometry']]
    parking_points = traffic_a[traffic_a['fclass'] == 'parking'][['osm_id', 'geometry']]
    crossing_points = traffic[traffic['fclass'] == 'crossing'][['osm_id', 'geometry']]
    bus_stop_points = transport_a[transport_a['fclass'] == 'bus_stop'][['osm_id', 'geometry']]
    
    # unnecessery dfs
    del buildings, landuse, pois_a, railways, roads, traffic_a, traffic, transport_a; gc.collect()
    
    # changing crs
    buildings_points = buildings_points.to_crs("epsg:4258")
    shop_points = shop_points.to_crs("epsg:4258")
    parks_points = parks_points.to_crs("epsg:4258")
    forest_points = forest_points.to_crs("epsg:4258")
    schools_points = schools_points.to_crs("epsg:4258")
    railways_points = railways_points.to_crs("epsg:4258")
    cycleways_points = cycleways_points.to_crs("epsg:4258")
    parking_points = parking_points.to_crs("epsg:4258")
    crossing_points = crossing_points.to_crs("epsg:4258")
    bus_stop_points = bus_stop_points.to_crs("epsg:4258")
    
    # list of dataframes
    list_of_dfs = [buildings_points, shop_points, parks_points, forest_points, schools_points, railways_points,
                  cycleways_points, parking_points, crossing_points, bus_stop_points]
    
    # names of new columns
    names = ['buildings', 'shops', 'parks', 'forests', 'schools', 'railways',
                  'cycleways', 'parkings', 'crossings', 'bus_stops']
    
    # groupby points in a loop
    for i in range(len(list_of_dfs)):
        actual_point = gpd.sjoin(list_of_dfs[i], df_output, how="inner", op="intersects")
        x = actual_point[['osm_id', 'buffer_index']].groupby(['buffer_index']).count()
        x.rename(columns={"osm_id": names[i]}, inplace=True)
        x.reset_index(inplace = True)
        df_output = df_output.merge(x, on = 'buffer_index', how='outer')
        
    df_output.fillna(0, inplace=True)
    df_output.drop(columns = {'jpt_nazwa_'}, inplace = True)

    return df_output

## Saving final dataset for buffers

In [36]:
df_krakow_inpost = merge_osm(inpost_buffers, 'powiat Kraków', '../datasets/raw_data/osm_malopolskie/')
df_krakow_inpost.to_csv('../datasets/preprocessed_data/df_krakow_gwr.csv', index = False)

In [38]:
del df_krakow_inpost; gc.collect();

In [42]:
df_warszawa_inpost = merge_osm(inpost_buffers, 'powiat Warszawa', '../datasets/raw_data/osm_mazowieckie/')
df_warszawa_inpost.to_csv('../datasets/preprocessed_data/df_warszawa_gwr.csv', index = False)

In [43]:
del df_warszawa_inpost; gc.collect();